In [1]:
# import requests
# import time
from pprint import pprint

import spotipy
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd
import numpy as np
import os

In [2]:
def get_playlist_tracks(playlist_id):
    playlist_items = sp.playlist_items(playlist_id)
    tracks = playlist_items['items']
    while playlist_items['next']:
        playlist_items = sp.next(playlist_items)
        tracks.extend(playlist_items['items'])
    return tracks

def get_track_fields(id_list):
    audio_features = sp.audio_features(id_list[:100])
    audio_features.extend(sp.audio_features(id_list[100:200]))
    audio_features.extend(sp.audio_features(id_list[200:]))
    return audio_features

In [16]:
CID = "7b80e1b5abf2415e81b6c03ecf926b77"
CSECRET = "7df2130db92849fc958d44f2c9187437"
RURI = "http://localhost/"
SCOPE = "user-library-read"

PLAYLIST_ID = "6Tmpg47Lqkba4kmDY7ymLC"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = CID,
                                               client_secret = CSECRET,
                                               redirect_uri = RURI,
                                               scope = SCOPE))
all_tracks = get_playlist_tracks(PLAYLIST_ID)

In [42]:
album_id = []
album_title = []
album_rd = []
album_seq = []
track_id = []
track_name = []
track_popularity = []

for i in range(len(all_tracks)):
    album_id.append(all_tracks[i]['track']['album']['id'])
    album_title.append(all_tracks[i]['track']['album']['name'])
    album_rd.append(all_tracks[i]['track']['album']['release_date'])
    album_seq.append(all_tracks[i]['track']['track_number'])
    track_id.append(all_tracks[i]['track']['id'])
    track_name.append(all_tracks[i]['track']['name'])
    track_popularity.append(all_tracks[i]['track']['popularity'])

audio_features = get_track_fields(track_id)

df = pd.DataFrame(list(zip(album_id, album_title, album_rd, album_seq, 
                           track_id, track_name, track_popularity, audio_features)), 
                  columns =['album_id', 'album_title', 'album_rd', 'album_seq',
                            'track_id','track_name','track_popularity','audio_features'])

df_audio = df['audio_features'].apply(pd.Series)
df = df.drop('audio_features', axis = 1)
final = df.join(df_audio, how='left')
final.to_csv('bts_song_features_224.csv')